In [8]:
# =============================
# Upload dataset
# =============================
from google.colab import files
files.upload()

# =============================
# Import libraries
# =============================
!pip install gradio -q

import pandas as pd
from sklearn.linear_model import LinearRegression
import gradio as gr
import matplotlib.pyplot as plt

# =============================
# Load dataset
# =============================
df = pd.read_csv("Dataset.csv", engine="python", on_bad_lines="skip")

# =============================
# Cleaning
# =============================
df["Arrival_time"] = pd.to_datetime(df["Arrival_time"], errors="coerce")
df["Departure_Time"] = pd.to_datetime(df["Departure_Time"], errors="coerce")

# =============================
# Feature creation
# =============================
journey = df.groupby("Train_No").agg(
    start=("Departure_Time","first"),
    end=("Arrival_time","last"),
    total_distance=("Distance","max"),
    stops=("Station_Name","count")
)

journey["duration"] = (journey["end"]-journey["start"]).dt.total_seconds()/3600
journey = journey.dropna()

# =============================
# Train model
# =============================
X = journey[["total_distance","stops"]]
y = journey["duration"]

model = LinearRegression().fit(X,y)

print("Model ready ✅")

# =============================
# UI function
# =============================
def predict(distance, stops):

    duration = model.predict([[distance,stops]])[0]

    # Create professional plot
    fig = plt.figure(figsize=(6,4))

    # Existing journey trend
    plt.scatter(journey["total_distance"], journey["duration"], alpha=0.3, label="Training Data")

    # User point
    plt.scatter(distance, duration, s=200, marker="*", label="Your Prediction")

    plt.xlabel("Distance")
    plt.ylabel("Duration")
    plt.title("Distance vs Duration Prediction")
    plt.legend()

    return duration, fig

# =============================
# Build UI
# =============================
ui = gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label="Distance (km)"),
        gr.Number(label="Stops")
    ],
    outputs=[
        gr.Number(label="Duration (hours)"),
        gr.Plot()
    ],
    title="🚆 Train Journey Duration Predictor",
    description="Enter distance and stops to predict duration"
)

ui.launch()

Saving Dataset.csv to Dataset (1).csv


/tmp/ipython-input-810037488.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Arrival_time"] = pd.to_datetime(df["Arrival_time"], errors="coerce")
/tmp/ipython-input-810037488.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Departure_Time"] = pd.to_datetime(df["Departure_Time"], errors="coerce")


Model ready ✅
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2c47d3924154392a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
